In [2]:
import pandas as pd
import re
import os
from datetime import datetime
import glob
import concurrent.futures
import time
import logging
import warnings
from openpyxl.styles import stylesheet

# Suppress openpyxl default style warning
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.styles.stylesheet")

# Configure logging: set level to DEBUG for detailed logging (change to INFO in production)
logging.basicConfig(
    level=logging.DEBUG,  # DEBUG logs detailed info; change to INFO for less verbosity
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('vehicle_data_processing.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

def safe_int_conversion(val):
    """
    Safely convert a value to an integer.
    
    Attempts to convert the input to float then int.
    If conversion fails, returns 0.
    """
    try:
        return int(float(val))
    except (ValueError, TypeError) as e:
        logger.debug(f"Conversion error for value '{val}': {e}")
        return 0

def extract_metadata_from_title_and_filename(title, file_name):
    """
    Extract RTO name and state from the given title and file name.
    
    Uses multiple regular expression patterns to try to extract:
      - The RTO name (e.g., "NAMSAI - AR20")
      - The state (defaulting to "Arunachal Pradesh" if applicable)
    
    Parameters:
        title (str): The title extracted from the Excel file.
        file_name (str): The name of the Excel file.
    
    Returns:
        tuple: (rto_name, state_name)
    """
    # Remove file extension for filename-based extraction
    base_file_name = os.path.splitext(file_name)[0]
    
    # Option 1: Look for pattern "of <RTO>, <State> ("
    match = re.search(r'of\s+([\w-]+)\s*,\s*(.+?)\s*\(', title)
    if match:
        logger.debug("Option 1 metadata extraction succeeded.")
        return match.group(1), match.group(2)
    
    # Option 2: If "of" exists, attempt capturing "of <RTO - ARXX>"
    if "of" in title:
        match = re.search(r'of\s+([\w\s]+-\s*AR\d+)', title)
        if match:
            logger.debug("Option 2 metadata extraction succeeded.")
            return match.group(1).strip(), "Arunachal Pradesh"
    
    # Option 3: Look for pattern "<RTO> - <ARXX>" in the title
    match = re.search(r'([\w\s]+)\s*-\s*(AR\d+)', title)
    if match:
        logger.debug("Option 3 metadata extraction succeeded.")
        return f"{match.group(1).strip()} - {match.group(2)}", "Arunachal Pradesh"
    
    # Option 4: Look in the filename for pattern "<RTO> - <ARXX>"
    match = re.search(r'([\w\s]+)\s*-\s*(AR\d+)', base_file_name)
    if match:
        logger.debug("Option 4 metadata extraction succeeded.")
        return f"{match.group(1).strip()} - {match.group(2)}", "Arunachal Pradesh"
    
    # Option 5: Use underscore-separated file name parts
    parts = base_file_name.split('_')
    if len(parts) > 1 and parts[0] and len(parts[0]) > 3:
        if "arunachal" in parts[0].lower():
            state_name = "Arunachal Pradesh"
        else:
            state_name = parts[0].replace('_', ' ').strip()
        
        rto_match = re.search(r'(AR\d+)', parts[1] if len(parts) > 1 else '')
        if rto_match:
            logger.debug("Option 5 metadata extraction succeeded.")
            return f"RTO Code {rto_match.group(1)}", state_name
        else:
            # Option 6: Use specific keyword matching from the title
            rto_pattern = re.search(
                r'(ITANAGAR|TAWANG|BOMDILA|SEPPA|ZIRO|DAPORIJO|PASIGHAT|YINGKIONG|TEZU|NAMSAI|CHANGLANG|KHONSA|LONGDING|DOIMUKH|LIKABALI|AALO|PANGIN|YUPIA)',
                title
            )
            if rto_pattern:
                logger.debug("Option 6 metadata extraction succeeded.")
                return rto_pattern.group(1), "Arunachal Pradesh"
    
    # Final fallback: Check if "arunachal" appears in file name
    if "arunachal" in file_name.lower():
        logger.debug("Final fallback metadata extraction used.")
        return "Unknown RTO (Arunachal Pradesh)", "Arunachal Pradesh"
    
    logger.debug("Metadata extraction failed; returning defaults.")
    return "Unknown RTO", "Unknown State"

def process_vehicle_data_file(input_file_path, min_nonzero_rows=1):
    """
    Process a single Excel file and return a DataFrame of the processed data.
    
    Reads the Excel file, extracts metadata, identifies the header row,
    processes the data rows, and applies filtering based on non-zero values.
    
    Parameters:
        input_file_path (str): The full path to the Excel file.
        min_nonzero_rows (int): Minimum required non-zero rows for the file to be valid.
    
    Returns:
        pd.DataFrame or None: The processed data or None if processing fails.
    """
    try:
        file_name = os.path.basename(input_file_path)
        logger.info(f"Processing file: {file_name}")
        
        # Read the Excel file without a header row.
        try:
            df = pd.read_excel(input_file_path, header=None, engine='openpyxl')
            logger.debug(f"File '{file_name}' read successfully. Data shape: {df.shape}")
        except Exception as e:
            logger.error(f"Failed to read {file_name}: {e}")
            return None
        
        # Extract metadata (e.g., RTO name and state) from the first cell or fallback to file name.
        title = str(df.iloc[0, 0]) if len(df) > 0 and not pd.isna(df.iloc[0, 0]) else file_name
        rto_name, state_name = extract_metadata_from_title_and_filename(title, file_name)
        logger.info(f"Extracted metadata for '{file_name}': RTO='{rto_name}', State='{state_name}'")
        
        # Identify the header row by scanning the first 15 rows for JAN and FEB columns.
        header_row = None
        for i in range(min(15, len(df))):
            row_data = df.iloc[i].astype(str).str.upper()
            if (('JAN' in row_data.values and 'FEB' in row_data.values) or
                ('JANUARY' in row_data.values and 'FEBRUARY' in row_data.values)):
                header_row = i
                break
        if header_row is None:
            logger.warning(f"Header row with JAN and FEB not found in '{file_name}'.")
            return None
        
        logger.debug(f"Header row detected at row {header_row} in '{file_name}': {df.iloc[header_row].tolist()}")
        
        # Determine column indices for JAN, FEB, and Vehicle Class
        headers = df.iloc[header_row].astype(str).str.upper()
        jan_col = feb_col = vehicle_class_col = None
        for i, val in enumerate(headers):
            if pd.isna(val):
                continue
            if val in ('JAN', 'JANUARY'):
                jan_col = i
            elif val in ('FEB', 'FEBRUARY'):
                feb_col = i
            elif 'VEHICLE CLASS' in val or ('VEHICLE' in val and 'CLASS' in val):
                vehicle_class_col = i
        
        # Fallback for vehicle class column: default to index 1 if not found.
        if vehicle_class_col is None:
            vehicle_class_col = 1
            vehicle_keywords = ['VEHICLE', 'CLASS', 'CAR', 'BUS', 'TRUCK', 'AUTO']
            for i, val in enumerate(headers):
                if isinstance(val, str) and any(keyword in val for keyword in vehicle_keywords):
                    vehicle_class_col = i
                    break
        
        if jan_col is None or feb_col is None or vehicle_class_col is None:
            logger.warning(f"Missing critical columns in '{file_name}'. Found - Vehicle Class: {vehicle_class_col}, Jan: {jan_col}, Feb: {feb_col}")
            return None
        
        # Process data rows (starting from the row after the header)
        data = []
        data_start_row = header_row + 1
        for i in range(data_start_row, len(df)):
            if vehicle_class_col >= len(df.columns) or pd.isna(df.iloc[i, vehicle_class_col]) or df.iloc[i].isna().all():
                continue
            
            try:
                vehicle_class = str(df.iloc[i, vehicle_class_col]).strip()
                if not vehicle_class:
                    continue
                
                raw_jan = df.iloc[i, jan_col] if jan_col < len(df.columns) else None
                raw_feb = df.iloc[i, feb_col] if feb_col < len(df.columns) else None
                logger.debug(f"File '{file_name}' - Row {i}: Raw JAN = {raw_jan}, Raw FEB = {raw_feb}")
                
                jan_value = safe_int_conversion(raw_jan)
                feb_value = safe_int_conversion(raw_feb)
                
                data.append({
                    'State': state_name,
                    'RTO Name': rto_name,
                    'Vehicle Class': vehicle_class,
                    'Jan': jan_value,
                    'Feb': feb_value
                })
            except Exception as row_error:
                logger.error(f"Error processing row {i} in '{file_name}': {row_error}")
                continue
        
        if not data:
            logger.warning(f"No valid data extracted from '{file_name}'.")
            return None
        
        # Optionally filter out rows where both Jan and Feb are zero
        if min_nonzero_rows > 0:
            nonzero_data = [row for row in data if row['Jan'] > 0 or row['Feb'] > 0]
            if len(nonzero_data) < min_nonzero_rows:
                logger.info(f"Skipping '{file_name}' - only {len(nonzero_data)} non-zero rows found (minimum required: {min_nonzero_rows}).")
                return None
            data = nonzero_data
        
        result_df = pd.DataFrame(data)
        logger.info(f"File '{file_name}' processed successfully with {len(result_df)} rows.")
        return result_df
    
    except Exception as e:
        logger.error(f"Unexpected error processing '{os.path.basename(input_file_path)}': {e}")
        return None

def batch_process_files(input_folder, output_folder, file_pattern="*.xlsx", max_workers=4, min_nonzero_rows=1):
    """
    Process all Excel files in the specified folder and save a single combined output Excel file.
    
    Parameters:
        input_folder (str): Directory containing the input Excel files.
        output_folder (str): Directory to save the combined output file.
        file_pattern (str): Pattern for matching Excel files.
        max_workers (int): Maximum number of parallel threads.
        min_nonzero_rows (int): Minimum required non-zero rows for a file to be considered valid.
    """
    try:
        os.makedirs(output_folder, exist_ok=True)
        logger.debug(f"Output folder '{output_folder}' is ready.")
    except Exception as e:
        logger.error(f"Failed to create output folder '{output_folder}': {e}")
        return
    
    file_paths = glob.glob(os.path.join(input_folder, file_pattern))
    total_files = len(file_paths)
    
    if total_files == 0:
        logger.warning(f"No files matching '{file_pattern}' found in '{input_folder}'.")
        return
    
    logger.info(f"Found {total_files} files to process in '{input_folder}'.")
    
    all_results = []
    processed_count = 0
    error_count = 0
    start_time = time.time()
    
    # Process files in parallel using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_file = {
            executor.submit(process_vehicle_data_file, file_path, min_nonzero_rows): file_path 
            for file_path in file_paths
        }
        for future in concurrent.futures.as_completed(future_to_file):
            file_path = future_to_file[future]
            try:
                result_df = future.result()
                if result_df is not None:
                    all_results.append(result_df)
                    processed_count += 1
                else:
                    error_count += 1
            except Exception as e:
                logger.error(f"Error processing '{os.path.basename(file_path)}': {e}")
                error_count += 1
    
    if all_results:
        try:
            combined_df = pd.concat(all_results, ignore_index=True)
        except Exception as e:
            logger.error(f"Error concatenating data frames: {e}")
            return
        
        # Create the output file name using today's date in DD-MM-YYYY format.
        today_str = datetime.now().strftime("%d-%m-%Y")
        combined_output = os.path.join(output_folder, f"Consolidated Maker Wise Data ({today_str}).xlsx")
        try:
            combined_df.to_excel(combined_output, index=False)
            logger.info(f"Combined output successfully saved to '{combined_output}'.")
        except Exception as e:
            logger.error(f"Failed to save combined output: {e}")
            return
        
        end_time = time.time()
        duration = end_time - start_time
        
        logger.info("Processing Summary:")
        logger.info(f"  Total files processed: {total_files}")
        logger.info(f"  Successfully processed: {processed_count}")
        logger.info(f"  Files with errors/skipped: {error_count}")
        logger.info(f"  Total records combined: {len(combined_df)}")
        logger.info(f"  Total processing time: {duration:.2f} seconds")
        
        nonzero_records = combined_df[(combined_df['Jan'] > 0) | (combined_df['Feb'] > 0)]
        logger.info(f"  Records with non-zero values: {len(nonzero_records)}")
        logger.info("  Top 5 vehicle classes by count:")
        logger.info(combined_df['Vehicle Class'].value_counts().head(5).to_string())
        
        if 'State' in combined_df.columns:
            logger.info("  Records by state:")
            logger.info(combined_df['State'].value_counts().to_string())
    else:
        logger.warning("No valid data was processed from any files.")

if __name__ == "__main__":
    # User-friendly start message
    print("Starting the Vehicle Data Processing script...")
    logger.info("Vehicle Data Processing Script started.")
    
    # Set your input and output folders (update these paths as needed)
    input_folder = r"C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh"
    output_folder = r"C:\Users\ASUS\OneDrive\Desktop"
    
    # Configuration parameters
    file_pattern = "*.xlsx"     # Process all Excel files in the input folder
    max_workers = 4             # Number of parallel threads
    min_nonzero_rows = 2        # Minimum non-zero rows required for a file to be processed
    
    try:
        batch_process_files(input_folder, output_folder, file_pattern, max_workers, min_nonzero_rows)
        logger.info("Vehicle Data Processing completed successfully.")
        print("Processing completed successfully. Check the output folder and log file for details.")
    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")
        print(f"An error occurred during processing: {e}")


2025-02-17 13:12:19,546 - INFO - Vehicle Data Processing Script started.
2025-02-17 13:12:19,547 - DEBUG - Output folder 'C:\Users\ASUS\OneDrive\Desktop' is ready.
2025-02-17 13:12:19,548 - INFO - Found 28 files to process in 'C:\Users\ASUS\OneDrive\Documents\Maker Wise\Arunachal Pradesh'.
2025-02-17 13:12:19,550 - INFO - Processing file: ANJAW - AR17.xlsx
2025-02-17 13:12:19,554 - INFO - Processing file: _1.xlsx
2025-02-17 13:12:19,557 - INFO - Processing file: _10.xlsx
2025-02-17 13:12:19,559 - INFO - Processing file: _11.xlsx
2025-02-17 13:12:19,572 - DEBUG - File 'ANJAW - AR17.xlsx' read successfully. Data shape: (6, 5)
2025-02-17 13:12:19,575 - DEBUG - Option 2 metadata extraction succeeded.
2025-02-17 13:12:19,577 - INFO - Extracted metadata for 'ANJAW - AR17.xlsx': RTO='ANJAW - AR17', State='Arunachal Pradesh'
2025-02-17 13:12:19,580 - DEBUG - Header row detected at row 3 in 'ANJAW - AR17.xlsx': [nan, nan, 'JAN', 'FEB', nan]
2025-02-17 13:12:19,582 - DEBUG - File '_1.xlsx' read 

Starting the Vehicle Data Processing script...


2025-02-17 13:12:19,742 - DEBUG - File '_19.xlsx' - Row 5: Raw JAN = 0, Raw FEB = 1
2025-02-17 13:12:19,742 - DEBUG - Option 2 metadata extraction succeeded.
2025-02-17 13:12:19,747 - DEBUG - File '_20.xlsx' read successfully. Data shape: (4, 5)
2025-02-17 13:12:19,748 - DEBUG - File '_18.xlsx' - Row 6: Raw JAN = 4, Raw FEB = 0
2025-02-17 13:12:19,748 - DEBUG - File '_19.xlsx' - Row 6: Raw JAN = 8, Raw FEB = 5
2025-02-17 13:12:19,749 - INFO - Extracted metadata for '_2.xlsx': RTO='DIBANG VALLEY - AR10', State='Arunachal Pradesh'
2025-02-17 13:12:19,749 - DEBUG - Option 2 metadata extraction succeeded.
2025-02-17 13:12:19,750 - INFO - File '_18.xlsx' processed successfully with 3 rows.
2025-02-17 13:12:19,751 - DEBUG - File '_19.xlsx' - Row 7: Raw JAN = 1, Raw FEB = 0
2025-02-17 13:12:19,752 - DEBUG - Header row detected at row 3 in '_2.xlsx': [nan, nan, 'JAN', 'FEB', nan]
2025-02-17 13:12:19,752 - INFO - Extracted metadata for '_20.xlsx': RTO='STATE TRANSPORT AUTHORITY - AR99', State='

Processing completed successfully. Check the output folder and log file for details.
